In [185]:
import numpy as np
import pandas as pd


class TOPSISmodel:
    """TOPSIS模型"""
    def __init__(self, A, w, small, medium, ranges):
        """
        初始化原始矩阵A,因素权重w,
        成本型索引small,中间型索引medium,
        区间型索引ranges"""
        self.A = A.copy()
        self.w = w.copy()
        self.small = small
        self.medium = medium
        self.ranges = ranges

    def start(self):
        """开始计算"""
        print("原始矩阵:")
        print(
            pd.DataFrame(self.A,
                         index=[f"方案{i+1}" for i in range(self.A.shape[0])],
                         columns=[f"因素{i+1}" for i in range(self.A.shape[1])]))
        self.__small2big()
        self.__medium2big()
        self.__range2big()
        print("\n正向化后:")
        print(
            pd.DataFrame(self.A,
                         index=[f"方案{i+1}" for i in range(self.A.shape[0])],
                         columns=[f"因素{i+1}" for i in range(self.A.shape[1])]))
        self.__normlize()
        print("\n规范化后:")
        print(
            pd.DataFrame(self.A,
                         index=[f"方案{i+1}" for i in range(self.A.shape[0])],
                         columns=[f"因素{i+1}" for i in range(self.A.shape[1])]))
        self.__get_scores()
        print(f"\n理想最优点:{self.best}")
        print(f"\n理想最劣点:{self.worst}")
        print("\n最终得分:")
        show_scores = self.scores.reshape(4)
        scores = pd.Series(show_scores,
                           index=[f"方案{i+1}" for i in range(self.A.shape[0])])
        print(scores)

    def __small2big(self):
        """成本型正向化"""
        for small_index in self.small:
            x = self.A[:, small_index - 1]
            self.A[:, small_index - 1] = max(x) - x

    def __medium2big(self):
        """中间型正向化"""
        n = self.medium.shape[0]
        for i in range(n):
            medium_index = self.medium[i, 0]
            x0 = self.medium[i, 1]
            x = self.A[:, medium_index - 1]
            x = np.abs(x - x0)
            x = 1 - x / np.max(x)
            self.A[:, medium_index - 1] = x

    def __range2big(self):
        """区间型正向化"""
        n = self.ranges.shape[0]
        for i in range(n):
            range_index = self.ranges[i, 0]
            a = self.ranges[i, 1]
            b = self.ranges[i, 2]
            x = self.A[:, range_index - 1]
            M = max(a - np.min(x), np.max(x) - b)
            for j in range(np.size(x)):
                if x[j] < a:
                    x[j] = 1 - (a - x[j]) / M
                elif a <= x[j] <= b:
                    x[j] = 1
                else:
                    x[j] = 1 - (x[j] - b) / M
            self.A[:, range_index - 1] = np.around(x, 4)

    def __normlize(self):
        """规范化"""
        for j in range(self.A.shape[1]):
            x = self.A[:, j]
            self.A[:, j] = np.around(x / np.linalg.norm(x), 4)

    def __get_scores(self):
        """计算得分并归一化"""
        self.best = np.max(self.A, axis=0)
        self.worst = np.min(self.A, axis=0)
        self.scores = []
        m, n = self.A.shape
        for i in range(m):
            d_best = d_worst = 0
            for j in range(n):
                d_best += self.w[:, j] * (self.A[i, j] - self.best[j])**2
                d_worst += self.w[:, j] * (self.A[i, j] - self.worst[j])**2
            d_best = np.sqrt(d_best)
            d_worst = np.sqrt(d_worst)
            score = d_worst / (d_best + d_worst)
            self.scores.append(score)

        # 归一化
        self.scores = np.array(self.scores)
        self.scores = np.around(self.scores / np.sum(self.scores), 4)

In [186]:
# 准备参数
A = np.array([[89, 2, 35.2, 6], [60, 0, 35.8, 7], [74, 1, 36.6, 8],
              [99, 3, 38.4, 9]])
w = np.array([[1, 1, 0, 0]])
small=np.array([[2]])
medium=np.array([[4,7]])
ranges=np.array([[3,36,37]])

# 建立模型并求解
pro=TOPSISmodel(A,w,small,medium,ranges)
pro.start()

原始矩阵:
      因素1  因素2   因素3  因素4
方案1  89.0  2.0  35.2  6.0
方案2  60.0  0.0  35.8  7.0
方案3  74.0  1.0  36.6  8.0
方案4  99.0  3.0  38.4  9.0

正向化后:
      因素1  因素2     因素3  因素4
方案1  89.0  1.0  0.4286  0.5
方案2  60.0  3.0  0.8571  1.0
方案3  74.0  2.0  1.0000  0.5
方案4  99.0  0.0  0.0000  0.0

规范化后:
        因素1     因素2     因素3     因素4
方案1  0.5437  0.2673  0.3095  0.4082
方案2  0.3665  0.8018  0.6188  0.8165
方案3  0.4520  0.5345  0.7220  0.4082
方案4  0.6048  0.0000  0.0000  0.0000

理想最优点:[0.6048 0.8018 0.722  0.8165]

理想最劣点:[0.3665 0.     0.     0.    ]

最终得分:
方案1    0.1857
方案2    0.3834
方案3    0.3170
方案4    0.1139
dtype: float64


0    [1, 2, 3]
dtype: object
